### Import

In [4]:
import pubmed

### Download Neurosurgery articles

In [2]:
db = 'pubmed'
query = 'text classification neural network'
query0 = 'neurosurgery'
query1 = '(neurosurg*) AND (complicat*[Text Word])'
query2 = '("Glioma/surgery"[Mesh] OR "Glioma/therapy"[Mesh])'
query3 = '("Neurosurgery"[Mesh] OR "Neurosurgical Procedures"[Mesh]) AND "Postoperative Complications"[Mesh]'
query4 = '("Neurosurgery"[Mesh] OR "Neurosurgical Procedures"[Mesh]) AND complicat*[Text Word]'

In [3]:
article_ids = pubmed.download_all_articles(query=query, db=db, refresh=False, cache=True)

242 articles found in pubmed with query specified.
142 articles are already stored in the database.
100 articles will be downloaded.



Total 242 articles stored in the database.


### Generate dataset

In [5]:
items = pubmed.generate_dataset(db, article_ids)
items.head()

,pmid,pmc,pii,mid,doi,elocation_id,language,title,authors,affiliations,...,publisher_location,publisher_nlm_id,publisher_issn_linking,abstract_len,abstract,copyright,mesh_quals_major,mesh_quals_minor,mesh_descriptors,keywords
0,32833649,None,None,None,10.1109/TNNLS.2020.3015830,10.1109/TNNLS.2020.3015830,eng,A Unified Framework for Multilingual Speech Re...,Lin Yi; Guo Dongyue; Zhang Jianwei; Chen Zheng...,Not defined // Not defined // Not defined // N...,...,United States,101616214,2162-237X,1665,This work focuses on robust speech recognition...,None,None,None,None,None
1,32818665,None,S1532-0464(20)30167-2,None,10.1016/j.jbi.2020.103539,S1532-0464(20)30167-2,eng,Comparing deep learning architectures for sent...,Colón-Ruiz Cristóbal; Segura-Bedmar Isabel,"Computer Science Department, University Carlos...",...,United States,100970413,1532-0464,1899,"Since the turn of the century, as millions of ...",Copyright © 2020. Published by Elsevier Inc.,None,None,None,Bidirectional Encoder Representations from Tra...
2,32814201,None,S1532-0464(20)30169-6,None,10.1016/j.jbi.2020.103541,S1532-0464(20)30169-6,eng,A Corpus-Driven Standardization Framework for ...,Peterson Kevin J; Jiang Guoqian; Liu Hongfang,"Department of Information Technology, Mayo Cli...",...,United States,100970413,1532-0464,1739,Free-text problem descriptions are brief expla...,Copyright © 2020. Published by Elsevier Inc.,None,None,None,Deep Learning (D000077321); Health Information...
3,32807454,None,S1052-3057(20)30460-2,None,10.1016/j.jstrokecerebrovasdis.2020.105042,S1052-3057(20)30460-2,eng,Toward automatic evaluation of medical abstrac...,Fischer Igor; Steiger Hans-Jakob,"Division of Informatics and Statistics, Depart...",...,United States,9111633,1052-3057,2092,BACKGROUND: Text mining with automatic extract...,Copyright © 2020 Elsevier Inc. All rights rese...,None,None,None,Artificial neural network; Machine learning; P...
4,32765261,PMC7378532,None,None,10.3389/fphar.2020.01028,10.3389/fphar.2020.01028,eng,Artificial Intelligence in Pharmacoepidemiolog...,Sessa Maurizio; Khan Abdul Rauf; Liang David; ...,"Department of Drug Design and Pharmacology, Un...",...,Switzerland,101548923,1663-9812,1496,Aim: To perform a systematic review on the app...,"Copyright © 2020 Sessa, Khan, Liang, Andersen ...",None,None,None,artificial intelligence; deep learning; machin...


In [18]:
items.info(verbose=False)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 242 entries, 0 to 241
Columns: 31 entries, pmid to keywords
dtypes: object(31)
memory usage: 60.5+ KB


In [33]:
items.to_csv('database/{0}_queryX.csv'.format(db), sep='|')
items.to_excel('database/{0}_queryX.xlsx'.format(db))

## MySQL tests

In [8]:
import pubmed
item = {}
filename = 'pubmed/29368597'

root = pubmed.get_element_tree(filename)
item = pubmed.parse_element_tree_pubmed(root)

In [4]:
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker

from config import conn_string_pubmed
from models.pubmed_model import PubmedArticle, BasePubmed

engine = create_engine(conn_string_pubmed)
BasePubmed.metadata.bind = engine
DBSession = sessionmaker(bind=engine)

session = DBSession()

### Add

In [9]:
article = PubmedArticle(item)
session.add(article) 
session.commit()

### Read

In [10]:
result = session.query(PubmedArticle).filter_by(pmid=18263039).first()
print(result)

[18263039] Title: A dynamic gesture recognition system for the Korean sign language (KSL).


### Update

In [34]:
item = session.query(PubmedArticle).filter_by(pmid=18263039).first()
item.pmc = 'PMCID888'
session.commit()

### Delete

In [5]:
article = session.query(PubmedArticle).filter_by(pmid=29368597).first()
session.delete(article)
session.commit()

In [5]:
articles = session.query(PubmedArticle).all()
for article in articles[:100]:
    session.delete(article)
session.commit()

In [11]:
session.close()